### VSAC for Kaggle - build Python moduls 

Source: https://github.com/ivashmak/vsac

Solvers for:
* Affine matrix (3 points)
* Homography matrix (4 points)
* Fundamental matrix (7 and 8 points)
* Essential matrix (5 points)
* Perspective projection matrix (3 points P3P, linear 6 points)

In [ ]:
!git clone https://github.com/ivashmak/vsac

In [ ]:
cd ./vsac

In [ ]:
!python -m pip install --upgrade pip

In [ ]:
%cp /kaggle/input/imcvsac/src/utils.cpp ./src/utils.cpp
%cp /kaggle/input/imcvsac/CMakeLists.txt .

In [ ]:
!git submodule update --init

In [ ]:
%mkdir build && cd build

In [ ]:
!sudo apt update

In [ ]:
!sudo apt -y install libopencv-dev python3-opencv

In [ ]:
!sudo apt install libeigen3-dev

In [ ]:
!sudo apt-get -y install libblas-dev liblapack-dev

In [ ]:
!sudo apt-get install cmake

In [ ]:
!sudo apt-get -y install gcc-10 g++-10

In [ ]:
!sudo apt-get -y install python3-dev

In [ ]:
!cmake . 

In [ ]:
!make -j $(nproc)

In [ ]:
%ls -al /kaggle/working/vsac/ 

In [ ]:
!pip install --upgrade setuptools

In [ ]:
%%writefile setup.py

#! /usr/bin/env python3
import os, re, sys, sysconfig, platform, subprocess

from distutils.version import LooseVersion
from setuptools import setup, Extension, find_packages
from setuptools.command.build_ext import build_ext
from setuptools.command.test import test as TestCommand
from shutil import copyfile, copymode

class CMakeExtension(Extension):
    def __init__(self, name, sourcedir=''):
        Extension.__init__(self, name, sources=[])
        self.sourcedir = os.path.abspath(sourcedir)

class CMakeBuild(build_ext):
    def run(self):
        try:
            out = subprocess.check_output(['cmake', '--version'])
        except OSError:
            raise RuntimeError(
                "CMake must be installed to build the following extensions: " +
                ", ".join(e.name for e in self.extensions))

        if platform.system() == "Windows":
            cmake_version = LooseVersion(re.search(r'version\s*([\d.]+)', out.decode()).group(1))
            if cmake_version < '3.1.0':
                raise RuntimeError("CMake >= 3.1.0 is required on Windows")

        for ext in self.extensions:
            self.build_extension(ext)

    def build_extension(self, ext):
        extdir = os.path.abspath(
            os.path.dirname(self.get_ext_fullpath(ext.name)))
        cmake_args = ['-DCMAKE_LIBRARY_OUTPUT_DIRECTORY=' + extdir,
                      '-DPYTHON_EXECUTABLE=' + sys.executable]

        cfg = 'Debug' if self.debug else 'Release'
        build_args = ['--config', cfg]

        if platform.system() == "Windows":
            cmake_args += ['-DCMAKE_LIBRARY_OUTPUT_DIRECTORY_{}={}'.format(
                cfg.upper(),
                extdir)]
            if sys.maxsize > 2**32:
                cmake_args += ['-A', 'x64']
            build_args += ['--', '/m']
        else:
            cmake_args += ['-DCMAKE_BUILD_TYPE=' + cfg]
            build_args += ['--', '-j8']

        env = os.environ.copy()
        env['CXXFLAGS'] = '{} -DVERSION_INFO=\\"{}\\"'.format(env.get('CXXFLAGS', ''),
            self.distribution.get_version())
        if not os.path.exists(self.build_temp):
            os.makedirs(self.build_temp)
        
        print(f'ALERT {self.build_temp}')

        # subprocess.check_call(['cmake', ext.sourcedir] + cmake_args,
        #                       cwd=self.build_temp, env=env)
        # subprocess.check_call(['cmake', '--build', '.'] + build_args,
        #                       cwd=self.build_temp)
        subprocess.check_call(['cmake', ext.sourcedir] + cmake_args, env=env)
        subprocess.check_call(['cmake', '--build', '.'] + build_args,)

requirements = ["numpy"]
from os import path
with open(path.join(path.abspath(path.dirname(__file__)), 'README.md')) as f:
    long_description = f.read()

setup(
    name='pvsac',
    version='0.1',
    author='Maksym Ivashechkin',
    author_email='maksimivashechkin@gmail.com',
    license='MIT',  
    url = 'https://github.com/ivashmak/vsac',   # Provide either the link to your github or to your website
    download_url = '',
    description='Robust estimator VSAC',
    long_description=long_description,
    long_description_content_type='text/markdown',
    packages=find_packages('python'),
    package_dir={'':'python'},
    ext_modules=[CMakeExtension('vsac')],
    extra_compile_args = ["-O3"], 
    cmdclass=dict(build_ext=CMakeBuild),
    #test_suite='tests',
    zip_safe=False,
    install_requires=requirements,
)

In [ ]:
!python setup.py install

In [ ]:
%ls -al ./dist

In [ ]:
!pip install wheel --upgrade

In [ ]:
!wheel convert ./dist/pvsac-0.1-py3.7-linux-x86_64.egg

In [ ]:
ls -al | grep pvsac

In [ ]:
from distutils import util
util.get_platform()

In [ ]:
!pip debug --verbose | grep cp37-none-

In [ ]:
%cp pvsac-0.1-py37-cp37-linux_x86_64.whl pvsac-0.1-cp37-none-linux_x86_64.whl

In [ ]:
!pip uninstall pvsac --y

In [ ]:
!pip install pvsac-0.1-cp37-none-linux_x86_64.whl

In [ ]:
import pvsac
from datetime import datetime
import cv2
import numpy as np

SAMPLE_PATH = "/kaggle/input/imcvsac/samples/data/"

points_file = open(f'{SAMPLE_PATH}leuven_pts.txt')

img1 = cv2.imread(f'{SAMPLE_PATH}leuvenA.jpg')
img2 = cv2.imread(f'{SAMPLE_PATH}leuvenB.jpg')

num_points = int(points_file.readline())
points = np.array([[float(num) for num in line.split(' ')] for line in points_file])
pts1, pts2 = points[:,0:2].copy(), points[:,2:4].copy()

def run(params, pts1, pts2, K1=None, K2=None, dist_coef1=None, dist_coef2=None, img1=None, img2=None):
    start_time = datetime.now()
    model, inliers = pvsac.estimate(params, pts1, pts2, K1, K2, dist_coef1, dist_coef2)
    runtime = datetime.now() - start_time
    print("Fundamental Matrix\n", model, "\n\n#inliers", inliers.sum(), "/", pts1.shape[0], 'time (ms)', runtime.microseconds/1e3)
    return model, inliers


params = pvsac.Params(pvsac.EstimationMethod.Fundamental, 
                      1., 0.99, 10000, 
                      pvsac.SamplingMethod.SAMPLING_UNIFORM, 
                      pvsac.ScoreMethod.SCORE_METHOD_MSAC)

F, inliers = run(params, pts1, pts2, img1=img1, img2=img2)